# Finetuning efficientNet

Lets try the model that is trending for image classification on HuggingFace: efficientnet_b2.ra_in1k

In [1]:
from datasets import load_dataset

cifar10dataset = load_dataset("cifar10", split="train")

In [2]:
cifar10dataset = cifar10dataset.train_test_split(test_size=0.2)

In [3]:
labels = cifar10dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [4]:
from transformers import AutoImageProcessor

#import timm
#model = timm.create_model("hf_hub:timm/efficientnet_b2.ra_in1k", pretrained=True)

checkpoint = "google/efficientnet-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

2023-08-14 11:17:24.073751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 11:17:25.039990: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [6]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["img"]]
    del examples["img"]
    return examples

In [7]:
cifar10dataset = cifar10dataset.with_transform(transforms)

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id,
)

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([10, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import torch 
torch.cuda.empty_cache()


In [13]:
training_args = TrainingArguments(
    output_dir="cifar10_efficientnet",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8, # memory error with 16
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=cifar10dataset["train"],
    eval_dataset=cifar10dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/mahnaz/mlprojects/bloom-classifier/venv_bloom-classifier/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3750 [00:00<?, ?it/s]

{'loss': 2.3033, 'learning_rate': 1.3333333333333334e-06, 'epoch': 0.01}
{'loss': 2.306, 'learning_rate': 2.666666666666667e-06, 'epoch': 0.02}
{'loss': 2.3034, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 2.3035, 'learning_rate': 5.333333333333334e-06, 'epoch': 0.03}
{'loss': 2.3012, 'learning_rate': 6.666666666666667e-06, 'epoch': 0.04}
{'loss': 2.3028, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 2.3027, 'learning_rate': 9.333333333333334e-06, 'epoch': 0.06}
{'loss': 2.3028, 'learning_rate': 1.0666666666666667e-05, 'epoch': 0.06}
{'loss': 2.3018, 'learning_rate': 1.2e-05, 'epoch': 0.07}
{'loss': 2.2992, 'learning_rate': 1.3333333333333333e-05, 'epoch': 0.08}
{'loss': 2.2992, 'learning_rate': 1.4666666666666668e-05, 'epoch': 0.09}
{'loss': 2.2995, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.1}
{'loss': 2.2997, 'learning_rate': 1.7333333333333336e-05, 'epoch': 0.1}
{'loss': 2.2959, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.11}
{'l

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.7926234006881714, 'eval_accuracy': 0.6835, 'eval_runtime': 40.1252, 'eval_samples_per_second': 249.22, 'eval_steps_per_second': 31.152, 'epoch': 1.0}
{'loss': 1.8565, 'learning_rate': 3.688888888888889e-05, 'epoch': 1.01}
{'loss': 1.8673, 'learning_rate': 3.674074074074074e-05, 'epoch': 1.02}
{'loss': 1.8376, 'learning_rate': 3.6592592592592596e-05, 'epoch': 1.02}
{'loss': 1.8554, 'learning_rate': 3.644444444444445e-05, 'epoch': 1.03}
{'loss': 1.8792, 'learning_rate': 3.62962962962963e-05, 'epoch': 1.04}
{'loss': 1.834, 'learning_rate': 3.614814814814815e-05, 'epoch': 1.05}
{'loss': 1.8633, 'learning_rate': 3.6e-05, 'epoch': 1.06}
{'loss': 1.8684, 'learning_rate': 3.5851851851851854e-05, 'epoch': 1.06}
{'loss': 1.8656, 'learning_rate': 3.570370370370371e-05, 'epoch': 1.07}
{'loss': 1.8156, 'learning_rate': 3.555555555555556e-05, 'epoch': 1.08}
{'loss': 1.8627, 'learning_rate': 3.540740740740741e-05, 'epoch': 1.09}
{'loss': 1.8759, 'learning_rate': 3.525925925925926e-05,

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.7647517919540405, 'eval_accuracy': 0.7112, 'eval_runtime': 39.7623, 'eval_samples_per_second': 251.495, 'eval_steps_per_second': 31.437, 'epoch': 2.0}
{'loss': 1.7673, 'learning_rate': 1.837037037037037e-05, 'epoch': 2.01}
{'loss': 1.7884, 'learning_rate': 1.8222222222222224e-05, 'epoch': 2.02}
{'loss': 1.7649, 'learning_rate': 1.8074074074074074e-05, 'epoch': 2.02}
{'loss': 1.8183, 'learning_rate': 1.7925925925925927e-05, 'epoch': 2.03}
{'loss': 1.8143, 'learning_rate': 1.777777777777778e-05, 'epoch': 2.04}
{'loss': 1.7723, 'learning_rate': 1.762962962962963e-05, 'epoch': 2.05}
{'loss': 1.8077, 'learning_rate': 1.7481481481481483e-05, 'epoch': 2.06}
{'loss': 1.7518, 'learning_rate': 1.7333333333333336e-05, 'epoch': 2.06}
{'loss': 1.7926, 'learning_rate': 1.7185185185185185e-05, 'epoch': 2.07}
{'loss': 1.8219, 'learning_rate': 1.7037037037037038e-05, 'epoch': 2.08}
{'loss': 1.7968, 'learning_rate': 1.688888888888889e-05, 'epoch': 2.09}
{'loss': 1.8083, 'learning_rate': 

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.7357195615768433, 'eval_accuracy': 0.7364, 'eval_runtime': 39.7567, 'eval_samples_per_second': 251.53, 'eval_steps_per_second': 31.441, 'epoch': 3.0}
{'train_runtime': 1539.76, 'train_samples_per_second': 77.934, 'train_steps_per_second': 2.435, 'train_loss': 1.8802003056844077, 'epoch': 3.0}


TrainOutput(global_step=3750, training_loss=1.8802003056844077, metrics={'train_runtime': 1539.76, 'train_samples_per_second': 77.934, 'train_steps_per_second': 2.435, 'train_loss': 1.8802003056844077, 'epoch': 3.0})